In [1]:
import numpy as np
import emg3d
import pygimli as pg
from emg3d.inversion.pygimli import Kernel, Inversion

In [2]:
survey = emg3d.Survey(emg3d.TxElectricPoint((1, 1, 1, 1, 1)), emg3d.RxElectricPoint((3, 3, 3, 3, 3)), 1)
grid = emg3d.TensorMesh([[1, 2, 3, 4], [2.5, 3.5], [1.1, 2.2, 3.3]])
model = emg3d.Model(grid, np.arange(1, grid.n_cells+1)*100, mapping='Conductivity')
sim = emg3d.Simulation(survey=survey, model=model)

In [26]:
markers = np.zeros(model.shape, order='F', dtype=int)
markers[::2, :, :] = 1
#markers[:, ::2, :] = 2
#markers[:, :, ::2] = 3
markers[0, 0, 0] = 2

In [27]:
fop = Kernel(simulation=sim, markers=markers)
INV = Inversion(fop=fop)

#INV.fop.setRegionProperties(0, background=True)
INV.fop.setRegionProperties(0, limits=(20, 30), startModel=25.0)
INV.fop.setRegionProperties(1, limits=(20, 30), startModel=25.0)
#INV.fop.setRegionProperties(0, single=True, limits=(100, 200), startModel=150)
INV.fop.setRegionProperties(2, fix=True, startModel=300)

In [28]:
model.property_x, markers

(array([[[ 100.,  900., 1700.],
         [ 500., 1300., 2100.]],
 
        [[ 200., 1000., 1800.],
         [ 600., 1400., 2200.]],
 
        [[ 300., 1100., 1900.],
         [ 700., 1500., 2300.]],
 
        [[ 400., 1200., 2000.],
         [ 800., 1600., 2400.]]]),
 array([[[2, 1, 1],
         [1, 1, 1]],
 
        [[0, 0, 0],
         [0, 0, 0]],
 
        [[1, 1, 1],
         [1, 1, 1]],
 
        [[0, 0, 0],
         [0, 0, 0]]]))

In [29]:
fop.model2emg3d(model.property_x.ravel('F')) #np.arange(1, model.size+1))#np.array([1, 1, 1, 1, 1, 1, 1.]))

09/07/24 - 09:27:03 - pyGIMLi - INFO - [False  True False  True False  True False  True False  True False  True
 False  True False  True False  True False  True False  True False  True]
09/07/24 - 09:27:03 - pyGIMLi - INFO - [ True False  True False  True False  True False False  True False  True
  True False  True False False  True False  True False  True False  True]
09/07/24 - 09:27:03 - pyGIMLi - INFO - [ 100.  200.  300.  400.  500.  600.  700.  800.  900. 1000. 1100. 1200.
 1300. 1400. 1500. 1600. 1700. 1800. 1900. 2000. 2100. 2200. 2300. 2400.]
09/07/24 - 09:27:03 - pyGIMLi - INFO - [False False  True False  True False  True False  True False  True False
  True False  True False  True False  True False  True False  True False]
09/07/24 - 09:27:03 - pyGIMLi - INFO - [False  True False  True False  True False  True False False  True False
 False  True False  True  True False  True False  True False  True False]
09/07/24 - 09:27:03 - pyGIMLi - INFO - [ 100.  200.  300.  400.  500. 

array([[[ 100.,  300., 2000.],
        [ 300.,  700., 2200.]],

       [[1300.,  500.,  900.],
        [1500., 1800., 1100.]],

       [[ 200., 1700., 2100.],
        [ 400.,  800., 2300.]],

       [[1400.,  600., 1000.],
        [1600., 1900., 1200.]]])

In [7]:
#fop.model2gimli(np.ones(model.shape))